In [1]:
import csv
import re
import os
import numpy as np
import pandas as pd
import 

In [2]:
from utils import *

In [3]:
os.getcwd()

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review'

In [30]:
def get_calibration_files(serial_nums,dirpath):
    """
    Function which gets all the calibration files associated with the
    instrument serial numbers.
    
    Args:
        serial_nums - serial numbers of the instruments
        dirpath - path to the directory containing the calibration files
    Returns:
        calibration_files - a dictionary of instrument uids with associated
            calibration files
    """
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)[0]
        sn = str(sn)
        files = []
        for file in os.listdir(dirpath):
            if sn in file:
                if 'Calibration_File' in file:
                    files.append(file)
                else:
                    pass
            else:
                pass
        
        calibration_files.update({uid:files})
        
    return calibration_files

In [12]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/NUTNRB/'

In [13]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [14]:
NUTNR = whoi_asset_tracking(spreadsheet=excel_spreadsheet,sheet_name=sheet_name,instrument_class='NUTNR',series='B')

In [15]:
NUTNR

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
875,NUTNR,B,239,115084,A00065,CGINS-NUTNRB-00239,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00004\n3305-00108-00048\n3305-00108...,3305-00308-00001,3305-00508-00040,3305-00900-00075\n3305-00900-00144\n3305-00900...,WH-SC12-5-NUTNR-1001,2012-11-13 00:00:00,GI01SUMO-00001\nCP04OSSM-00006,NaN,NSIF,Reading High nitrate levels during calibration...
876,NUTNR,B,240,115085,A00066,CGINS-NUTNRB-00240,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00003\n3305-00108-00029\n3305-00108...,NaN,NaN,3305-00900-00008\n3305-00900-00231\n3305-00900...,WH-SC12-5-NUTNR-1001,2012-11-13 00:00:00,CP01CNSM-00001\nGS01SUMO-00002,NaN,NSIF,"09/2017: Clock issue - resets back to Jan 1, 2..."
877,NUTNR,B,260,115671,A00383,CGINS-NUTNRB-00260,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00010\n3305-00108-00056,NaN,3305-00508-00010,3305-00900-00109\n3305-00900-00317,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,GI Spare\nCP1 spare\nGS01SUMO-00001\nCP04OSSM-...,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
878,NUTNR,B,261,115672,A00384,CGINS-NUTNRB-00261,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00011\n3305-00108-00021\n3305-00108...,3305-00308-00007\n3305-00308-00031\n3305-00308...,3305-00508-00022\n3305-00508-00041,3305-00900-00071\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,CP03ISSM-00002\nCP01CNSM-00005\nCP01CNSM-00007,NaN,NSIF,Returned to vendor 6/24/14 (RMA#2014-125)\nSen...
879,NUTNR,B,262,115673,A00385,CGINS-NUTNRB-00262,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00012\n3305-00108-00064,3305-00308-00002,NaN,3305-00900-00064\n3305-00900-00153\n3305-00900...,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,GI01SUMO-00001,NaN,NaN,Sent to vedor as part of trade in for new (SUN...
880,NUTNR,B,266,116564,A00880,CGINS-NUTNRB-00266,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00015\n3305-00108-00040\n3305-00108...,3305-00308-00004\n3305-00308-00032,3305-00508-00024,3305-00900-00036\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP04OSSM-00001\nCP04OSSM-00004\nCP01CNSM-00008,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
881,NUTNR,B,267,116562,A00878,CGINS-NUTNRB-00267,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00018\n3305-00108-00053,NaN,3305-00508-00009,3305-00900-00109\n3305-00900-00363,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,GS01SUMO-00001\nCP01CNSM-00006,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
882,NUTNR,B,268,116563,A00879,CGINS-NUTNRB-00268,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00017\n3305-00108-00046\n3305-00108...,3305-00308-00003\n3305-00308-00034,3305-00508-00023,3305-00900-00071\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP03ISSM-00001\nCP03ISSM-00004\nCP03ISSM-00006,NaN,NSIF,4/2016: Scheduled to sample every half hour. S...
883,NUTNR,B,269,116565,A00881,CGINS-NUTNRB-00269,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00016\n3305-00108-00047\n3305-00114...,3305-00308-00005\n3305-00308-00009,3305-00508-00001,3305-00900-00071\n3305-00900-00317,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP01CNSM-00002\nCP01CNSM-00003\nCP03ISSM-00005,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
884,NUTNR,B,270,116899,A01142,CGINS-NUTNRB-00270,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00020\n3305-00108-00050,3305-00308-00006\n3305-00308-00008,3305-00508-00038,3305-00900-00071\n3305-00900-00317,WH-SC12-05-NUTNR-1006,2014-09-24 00:00:00,CP3a Spare\nCP04OSSM-00002\nGI01SUMO-00003,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...


In [16]:
uids = list(set(NUTNR['UID']))
qct_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    NUTNR['UID_match'] = NUTNR['UID'].apply(lambda x: True if uid in x else False)
    qct_series = NUTNR[NUTNR['UID_match'] == True]['QCT Testing']
    qct_series = list(qct_series.iloc[0].split('\n'))
    qct_dict.update({uid:qct_series})

In [19]:
uids[0:5]

['CGINS-NUTNRB-00284',
 'CGINS-NUTNRB-01087',
 'CGINS-NUTNRB-00239',
 'CGINS-NUTNRB-00274',
 'CGINS-NUTNRB-01093']

In [26]:
serial_nums = get_serial_nums(NUTNR, uids)
serial_nums

{'CGINS-NUTNRB-00284': [284],
 'CGINS-NUTNRB-01087': ['NTR-1087'],
 'CGINS-NUTNRB-00239': [239],
 'CGINS-NUTNRB-00274': [274],
 'CGINS-NUTNRB-01093': ['NTR-1093'],
 'CGINS-NUTNRB-01063': ['NTR-1063'],
 'CGINS-NUTNRB-01062': ['NTR-1062'],
 'CGINS-NUTNRB-01086': ['NTR-1086'],
 'CGINS-NUTNRB-01090': ['NTR-1090'],
 'CGINS-NUTNRB-00240': [240],
 'CGINS-NUTNRB-00276': [276],
 'CGINS-NUTNRB-00261': [261],
 'CGINS-NUTNRB-00280': [280],
 'CGINS-NUTNRB-00266': [266],
 'CGINS-NUTNRB-01107': ['NTR-1107'],
 'CGINS-NUTNRB-00260': [260],
 'CGINS-NUTNRB-01064': ['NTR-1064'],
 'CGINS-NUTNRB-00267': [267],
 'CGINS-NUTNRB-01091': ['NTR-1091'],
 'CGINS-NUTNRB-01103': ['NTR-1103'],
 'CGINS-NUTNRB-01094': ['NTR-1094'],
 'CGINS-NUTNRB-00273': [273],
 'CGINS-NUTNRB-01106': ['NTR-1106'],
 'CGINS-NUTNRB-00277': [277],
 'CGINS-NUTNRB-01092': ['NTR-1092'],
 'CGINS-NUTNRB-00269': [269],
 'CGINS-NUTNRB-00283': [283],
 'CGINS-NUTNRB-01089': ['NTR-1089'],
 'CGINS-NUTNRB-01088': ['NTR-1088'],
 'CGINS-NUTNRB-00271': [2

In [31]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

{'CGINS-NUTNRB-00284': ['NUTNR-B_ISUS_SN_284_Calibration_Files_2015-08-10.zip',
  'NUTNR-B_ISUS_SN_284_Calibration_Files_2016-06-27.zip'],
 'CGINS-NUTNRB-01087': ['NUTNR-B_SUNA_SN_NTR-1087_Calibration_Files_2017-10-25.zip'],
 'CGINS-NUTNRB-00239': ['NUTNR-B_ISUS_SN_239_Calibration_Files _2012-11-30.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2012-10-08.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2016-02-29.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2016-09-30.zip'],
 'CGINS-NUTNRB-00274': ['NUTNR-B_ISUS_SN_274_Calibration_Files_2015-06-18.zip',
  'NUTNR-B_ISUS_SN_274_Calibration_Files_2016-11-09.zip'],
 'CGINS-NUTNRB-01093': ['NUTNR-B_SUNA_SN_NTR-1093_Calibration_Files_2017-10-27.zip'],
 'CGINS-NUTNRB-01063': ['NUTNR-B_SUNA_SN_NTR-1063_Calibration_Files_2017-09-05.zip',
  'NUTNR-B_SUNA_SN_NTR-1063_Calibration_Files_2018-12-20.zip'],
 'CGINS-NUTNRB-01062': ['NUTNR-B_SUNA_SN_NTR-1062_Calibration_Files_2017-09-02.zip'],
 'CGINS-NUTNRB-01086': ['NUTNR-B_SUNA_SN_NTR-1086_Calibrati

In [33]:
filename = cal_dict['CGINS-NUTNRB-01065'][1]
filename

'NUTNR-B_SUNA_SN_NTR-1065_Calibration_Files_2018-12-20.zip'

In [34]:
cal_directory

'/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/'

In [38]:
filepath = generate_file_path(cal_directory,filename.split('.')[0],ext=['.zip'])
filepath

'/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR-B_SUNA_SN_NTR-1065_Calibration_Files_2018-12-20.zip'

In [42]:
with ZipFile(filepath) as zfile:
    filename = [name for name in zfile.namelist() if name.endswith('.cal') and name.startswith('SNA')]
    if len(filename) == 1:
        data = zfile.read(filename[0]).decode('ASCII')
    elif len(filename) > 1:
        raise Exception(f'More than one calibration file found in {filepath}.')
    else:
        FileExistsError(f"No .cal file found in {filepath}")

In [43]:
data

'H,SUNA 1065 Cal A  extinction coefficients and reference spectra\r\nH,File generated by Internal Software Suite 1.9.23_30\r\nH,File format version 3\r\nH,File creation time 20-Dec-2018 16:34:10\r\nH,Operator production\r\nH,PATH_LENGTH 10\r\nH,INT_PERIOD 225\r\nH,CONC_CAL_NO3 40.00\r\nH,CONC_CAL_SWA 34.50\r\nH,T_S_CORRECTABLE\r\nH,T_CAL 20.00\r\nH,T_CAL_SWA 20.00\r\nH,NitrateFile SUNA_1065_001.xml\r\nH,DIW Log file SUNA1065_CAL03_DIW_1.raw\r\nH,LNSW Log file SUNA1065_CAL03_LNSW_1.raw\r\nH,Nitrate in LNSW Log file SUNA1065_CAL03_NO3_1.raw\r\nH,Wavelength,nm\r\nH,NITRATE,uM\r\nH,AUX1,none\r\nH,AUX2,none\r\nH,Reference,counts\r\nH,Wavelength,NO3,SWA,TSWA,Reference\r\nE,189.66,0.00818343,0.00516340,0.00568202,41.00\r\nE,190.46,-0.00352403,0.00724791,0.00782146,26.00\r\nE,191.26,99.00000000,-0.00024984,-0.00026439,35.00\r\nE,192.06,0.00107640,0.01262849,0.01310521,51.00\r\nE,192.87,0.00580891,0.00001158,0.00001179,42.00\r\nE,193.67,-0.00155535,0.00733214,0.00731534,56.00\r\nE,194.47,-0.006

In [45]:
for line in data.splitlines():
    print(line.split(','))

['H', 'SUNA 1065 Cal A  extinction coefficients and reference spectra']
['H', 'File generated by Internal Software Suite 1.9.23_30']
['H', 'File format version 3']
['H', 'File creation time 20-Dec-2018 16:34:10']
['H', 'Operator production']
['H', 'PATH_LENGTH 10']
['H', 'INT_PERIOD 225']
['H', 'CONC_CAL_NO3 40.00']
['H', 'CONC_CAL_SWA 34.50']
['H', 'T_S_CORRECTABLE']
['H', 'T_CAL 20.00']
['H', 'T_CAL_SWA 20.00']
['H', 'NitrateFile SUNA_1065_001.xml']
['H', 'DIW Log file SUNA1065_CAL03_DIW_1.raw']
['H', 'LNSW Log file SUNA1065_CAL03_LNSW_1.raw']
['H', 'Nitrate in LNSW Log file SUNA1065_CAL03_NO3_1.raw']
['H', 'Wavelength', 'nm']
['H', 'NITRATE', 'uM']
['H', 'AUX1', 'none']
['H', 'AUX2', 'none']
['H', 'Reference', 'counts']
['H', 'Wavelength', 'NO3', 'SWA', 'TSWA', 'Reference']
['E', '189.66', '0.00818343', '0.00516340', '0.00568202', '41.00']
['E', '190.46', '-0.00352403', '0.00724791', '0.00782146', '26.00']
['E', '191.26', '99.00000000', '-0.00024984', '-0.00026439', '35.00']
['E', '

In [56]:
serial = None
for line in data.splitlines():
    if line.startswith('H'):
        # Get the serial number
        if 'SUNA' in line and serial == None:
            _, info = line.split(',')
            serial = info.split()[1]
        # 

In [58]:
serial

'1065'